In [1]:
import numpy as np
import _dunsros as DR
from kan import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

ModuleNotFoundError: No module named '_dunsros'

In [ ]:
dr_corr = DR.DunsRos()

In [ ]:
k_og = 178              #отношение добычи газа к нефти
muw = 1.004 * 1e-3      #вязкость воды
muo = 0.97 * 1e-3       #вязкость нефти
d = 0.1524              #диаметр трубы
theta = 90              #угол наклона скважины
q0 = 1.59 * 1e3         #дебит нефти
bo = 1.197              #объемный коэффициент нефти
bw = 1                  #объемный коэффициент воды
bg = 9.1 * 1e-3         #объемный коэффициент газа
rs = 50.6               #астворимость газа в нефти (конденсации) или выделения газа из нефти (выкипания)
rsw = 11.4              #растворимость газа в воде — зависит от пластовых температуры и давления, необходимо уточнять
qg0 = 283 * 1e3         #дебит газа н.у.
rho0 = 8 * 1e2          #плотность нефти
rhog = 7 * 1e2          #плотность газ
rhow = 1 * 1e3          #плотность воды
sigmao = 8.41 * 1e-3    #коэффициент поверхностного натяжения нефти
sigmaw = 58.9 * 1e-3    #коэффициент поверхностного натяжения воды при 100 гр.Ц.
muo = 0.97 * 1e-3       #вязкость нефти
muw = 0.28 * 1e-3       #вязкость воды при 100 гр.Ц.
mug = 0.016 * 1e-3      #вязкость газа
epsilon = 18.288 * 1e-6 #шероховатость стенки трубы

wc = 1 - qg0 / q0 / k_og                                #коэффициент обводненности
qo = q0 * (1 - wc) * bo                                 #объемный дебит нефти
ql = ((1 - wc) * bo + wc * bw) * qo                     #дебит жидкости
qg = (qg0 - qo * rs - qo * wc * rsw) * bg               #расход газа
rho_lrc_kgm3 = rho0 * (1 - wc) / bo + rhow * wc / bw    #плотность жидкой фазы
rho_grc_kgm3 = rhog / rs                                #плотность газа
sigma_l_nm = sigmao * (1 - wc) + sigmaw * wc            #коэффициент поверхностного натяжения жидкость-газ
p = 14                                                  #текущее давление
mul_rc_cp = muo * (1 - wc)  + muw * wc                  #вязкость жидкости
Sd = np.pi * d**2 / 4                                   #площадь сечения трубы

sec_p_day = 24 * 60 * 60  #переведем все в систему СИ
qo = qo / sec_p_day
ql = ql / sec_p_day
qg = qg / sec_p_day


args_params = {
        "d": d,                         #диаметр в метрах
        "theta_deg": theta,             #угол
        "ql_rc_m3day": ql,              #дебит жидкости
        "qg_rc_m3day": qg,              #расход газа
        "rho_lrc_kgm3": rho_lrc_kgm3,   #плотность жидкой фазы
        "rho_grc_kgm3": rho_grc_kgm3,   #плотность газа
        "sigma_l_nm": sigma_l_nm,       #коэффициент поверхностного натяжения жидкость-газ
        "p": p,                         #текущее давление
        "mul_rc_cp": mul_rc_cp,         #вязкость жидкости
}



In [ ]:
calibr_C_grav = 1 #коэффициент калибровки гравитационной компоненты градиента
calibr_C_fric = 1 #коэффициент калибровки фрикционной компоненты градиента

dr_corr.calc_params(**args_params)

args_fric_grad = {
        "eps_m": epsilon,
        "mul_rc_cp": mul_rc_cp,
        "mug_rc_cp": mug,
        "c_calibr_fric": calibr_C_fric,
        "rho_lrc_kgm3": rho_lrc_kgm3,
        "rho_grc_kgm3": rho_grc_kgm3,
        "sigma_l_nm": sigma_l_nm,
}

dr_corr.calc_grav(theta, calibr_C_grav) + dr_corr.calc_fric(**args_fric_grad), dr_corr.calc_grav(theta, calibr_C_grav), dr_corr.calc_fric(**args_fric_grad)

(4246.693393278933, 4147.459405331203, 99.23398794773075)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# let's construct a dataset
f = lambda x: x[:,0]**2 + 0.3*x[:,1] + 0.1*x[:,2]**3 + 0.0*x[:,3]
dataset = create_dataset(f, n_var=4, device=device)

input_vars = [r'$x_'+str(i)+'$' for i in range(4)]

model = KAN(width=[4,5,1], device=device)
model.fit(dataset, steps=40, lamb=0.001);

NameError: name 'torch' is not defined